In [22]:
import os
import configparser
from langchain_groq import ChatGroq
from langchain_cohere import ChatCohere
from langchain_core.messages import HumanMessage, SystemMessage

from langchain.memory import ConversationBufferMemory 

# Configuration setup
config = configparser.ConfigParser()
config.read('/workspaces/langchainCDAC9nov/config.ini')
groq = config['groq']
cohere = config['cohere']

os.environ['GROQ_API_KEY'] = groq.get('GROQ_API_KEY')
os.environ['COHERE_API_KEY'] = cohere.get('COHERE_API_KEY')


In [27]:
store = {}
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory,
)
# Function to retrieve session-specific history or create a new one
def get_session_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()  # Create a new history store for this session
    return store[session_id]

# Function to simulate conversation and store messages
async def chat_with_model(model, session_id, prompt):
    # Get the session history
    conversation_history = get_session_history(session_id)

    # Add user input to the conversation history
    conversation_history.add_message(HumanMessage(content=prompt))
    
    # Get the model's response based on the session's history
    response = model.invoke([SystemMessage(content="You are a Bollywood storyteller."),
                             HumanMessage(content=prompt)])
    
    # Add model's response to conversation history
    conversation_history.add_message(SystemMessage(content=response.content))
    
    return response.content

In [28]:
# Random element pickers for actor, location, and theme
import random
def actor_picker():
    actors = ["Shah Rukh Khan", "Deepika Padukone", "Ranbir Kapoor", "Alia Bhatt"]
    return random.choice(actors)

def location_picker():
    locations = ["Shimla", "Mumbai", "Goa", "Jaipur"]
    return random.choice(locations)

def theme_picker():
    themes = ["Adventure", "Romance", "Thriller", "Drama"]
    return random.choice(themes)

In [29]:
# Functions to generate stories using different models
async def generate_story_cohere(actor, location, theme):
    session_id = "cohere_story"
    prompt = f"Create a Bollywood story starring {actor}, set in {location}, with a theme of {theme}."
    cohere_model = ChatCohere(model="command-r-plus")
    return await chat_with_model(cohere_model, session_id, prompt)

async def generate_story_groq(actor, location, theme):
    session_id = "groq_story"
    prompt = f"Create a Bollywood story starring {actor}, set in {location}, with a theme of {theme}."
    groq_model = ChatGroq(model="llama3-8b-8192")
    return await chat_with_model(groq_model, session_id, prompt)

In [30]:
# Randomly pick elements for the story
actor = actor_picker()
location = location_picker()
theme = theme_picker()

# Print selected elements
print(f"Selected Actor: {actor}")
print(f"Selected Location: {location}")
print(f"Selected Theme: {theme}")

Selected Actor: Alia Bhatt
Selected Location: Jaipur
Selected Theme: Drama


In [31]:
cohere_story = await generate_story_cohere(actor, location, theme)
print(f"\nStory from Cohere Model:\n{cohere_story}\n")

groq_story = await generate_story_groq(actor, location, theme)
print(f"\nStory from Groq Model:\n{groq_story}\n")


Story from Cohere Model:
Once upon a time, in the vibrant city of Jaipur, lived a young and talented artist named Alia. Alia Bhatt was an aspiring painter who dreamed of showcasing her artwork to the world. She spent her days capturing the rich culture and vibrant hues of Jaipur in her canvases. The bustling markets, the majestic palaces, and the lively people of the city inspired her immensely. 

However, Alia's life took an unexpected turn when she received news that her father, a renowned archaeologist, had gone missing while on an expedition to uncover ancient ruins outside the city. Determined to find her father, Alia embarked on a journey that took her deep into the mysterious and rugged terrain surrounding Jaipur. 

As she ventured forth, she encountered a world of secrets and intrigue. She discovered that her father's disappearance was linked to a long-forgotten rivalry between two powerful families, one of which held a deep connection to her own past. As Alia delved deeper, s

In [32]:
# Simulate user customization of the story
user_input = "Change the location to Goa and make it a romantic theme."
session_id = "session_123"  

In [33]:
# Get customized story responses
response_cohere = await chat_with_model(cohere_model, session_id, user_input)
print(f"\nModified Story from Cohere:\n{response_cohere}")


Modified Story from Cohere:
Once upon a time, in the vibrant and sunny state of Goa, there lived a young and free-spirited couple, Rahul and Priya. They were as different as chalk and cheese but shared an unwavering love for the breathtaking beaches and vibrant culture that Goa offered.

Rahul, a local Goan boy, was a surfer and a self-proclaimed beach bum. He spent his days riding the waves and charming the tourists with his good looks and easy-going personality. Priya, on the other hand, was an artist from the city, who had fallen in love with Goa's vibrant colors and laid-back lifestyle. She spent her days painting the beautiful landscapes and selling her artwork to the locals and tourists alike.

As fate would have it, Rahul and Priya met one fateful day on Calangute Beach. Priya was captivated by Rahul's surfing skills, and Rahul was instantly smitten by Priya's unique spirit and talent. They struck up a conversation, and soon realized they shared a deep connection and a love for

In [34]:
response_groq = await chat_with_model(groq_model, session_id, user_input)
print(f"\nModified Story from Groq:\n{response_groq}")


Modified Story from Groq:
The sun-kissed beaches of Goa, the perfect backdrop for a romantic tale of love and longing. Let me weave a story for you, a tale of two souls who find each other amidst the picturesque landscape of this beautiful state.

Meet Aarav, a young and charming hotelier who has inherited a quaint beachside resort in North Goa. He's a free spirit, always chasing the next big thrill, but deep down, he's searching for something more meaningful in life.

Enter Nalini, a stunning and talented photographer who has just moved to Goa to escape the monotony of city life. She's a hopeless romantic, always on the lookout for inspiration for her next big project. Her eyes are like the ocean, sparkling with secrets and stories waiting to be told.

Aarav and Nalini's paths cross at a quaint little café in Ashwem, where Aarav is trying to convince the owner to let him host a music festival at his resort. Nalini, sipping on a strong cup of coffee, is immediately drawn to Aarav's in

In [35]:
# Get customized story responses
response_groq = await chat_with_model(cohere_model, session_id, user_input)
print(f"\nModified Story from Cohere:\n{response_groq}")


Modified Story from Cohere:
Once upon a time, in the vibrant and picturesque state of Goa, there lived a young and charismatic man named Rohit. He was an aspiring musician, with a passion for playing the guitar and a dream to make it big in the world of music. One fateful day, while performing at a local beach cafe, he locked eyes with a beautiful and free-spirited woman named Sonali. She was an artist, with a love for painting and a heart that longed for adventure.

As their eyes met, something magical happened. It was as if time stood still and they were the only two people in the world. They felt an instant connection, a pull that drew them towards each other. From that day on, they became inseparable. They spent their days exploring the beautiful beaches of Goa, swimming in the Arabian Sea, and wandering through the colorful streets of Panaji. They shared their dreams, their passions, and their love for the arts.

Every sunset brought a new adventure. They would ride on Rohit's mo